# 1️⃣ Setup: 환경 설정 및 프로젝트 준비

이 노트북은 다음을 수행합니다:
1. 가상환경 연결 및 확인
2. Git Repository 클론
3. 데이터셋 다운로드 (DIV2K, 벤치마크)
4. CATANet-L x2 원본 모델 PTL 변환

## 1.1 가상환경 설정

In [ ]:
import os
import sys

# 가상환경 경로 설정
home_dir = os.path.expanduser('~')
venv_path = os.path.join(home_dir, 'my_ssl/ssl_env')
site_packages = os.path.join(venv_path, 'lib', 'python3.12', 'site-packages')

# 실행 파일 경로
python_exec = os.path.join(venv_path, 'bin', 'python')
pip_exec = os.path.join(venv_path, 'bin', 'pip')

# sys.path에 가상환경 추가
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)
    print(f"✅ 가상환경 라이브러리 경로 추가됨: {site_packages}")

# 가상환경 존재 여부 확인
if not os.path.exists(venv_path):
    print(f"🚀 가상환경 생성 중: {venv_path}")
    !python3 -m venv {venv_path}
    !{pip_exec} install --upgrade pip -q
    print("✅ 가상환경 생성 완료")
else:
    print(f"✅ 가상환경이 이미 존재합니다: {venv_path}")

print(f"🔹 Python: {python_exec}")

## 1.2 GPU 확인

In [ ]:
import torch
print(f"✅ PyTorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"✅ CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CUDA Not Available")

## 1.3 Git Repository 클론

In [ ]:
repo_url = "https://github.com/paaran2024/uos-ssl-project-ai.git"
branch = "hj"
project_dir = os.path.join(home_dir, 'my_ssl/uos-ssl-project/ai')

if os.path.exists(project_dir):
    print("✅ 폴더가 이미 존재합니다. 최신 코드로 업데이트합니다.")
    %cd {project_dir}
    !git fetch origin {branch}
    !git pull origin {branch}
else:
    print(f"🚀 Git 클론 중: {repo_url}")
    os.makedirs(os.path.join(home_dir, 'my_ssl/uos-ssl-project'), exist_ok=True)
    %cd {os.path.join(home_dir, 'my_ssl/uos-ssl-project')}
    !git clone -b {branch} {repo_url} ai
    print("✅ 클론 완료")

%cd {project_dir}
print(f"\n📁 현재 디렉토리: {os.getcwd()}")

## 1.4 필수 패키지 설치

In [ ]:
print("📦 필수 패키지 설치 중...")
!{pip_exec} install einops pandas numpy pyyaml tqdm opencv-python-headless scikit-image matplotlib scipy -q
print("✅ 패키지 설치 완료")

## 1.5 DIV2K 데이터셋 다운로드

In [ ]:
div2k_dir = os.path.join(project_dir, 'datasets/DIV2K')
os.makedirs(div2k_dir, exist_ok=True)

%cd {div2k_dir}

# HR 이미지
if not os.path.exists('DIV2K_train_HR'):
    print("📥 DIV2K HR 다운로드 중...")
    !wget -q http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
    !unzip -q DIV2K_train_HR.zip && rm DIV2K_train_HR.zip
    print("✅ HR 완료")
else:
    print("✅ DIV2K HR 이미 존재")

# LR 이미지 (x2)
if not os.path.exists('DIV2K_train_LR_bicubic'):
    print("📥 DIV2K LR (x2) 다운로드 중...")
    !wget -q http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
    !unzip -q DIV2K_train_LR_bicubic_X2.zip && rm DIV2K_train_LR_bicubic_X2.zip
    print("✅ LR 완료")
else:
    print("✅ DIV2K LR 이미 존재")

print(f"\n📁 DIV2K 구조:")
!ls -la

## 1.6 벤치마크 데이터셋 다운로드

In [ ]:
benchmark_dir = os.path.join(project_dir, 'datasets/benchmark')
os.makedirs(benchmark_dir, exist_ok=True)

%cd {benchmark_dir}

if not os.path.exists('Set5'):
    print("📥 벤치마크 데이터셋 다운로드 중...")
    print("   (Set5, Set14, BSD100, Urban100, Manga109)")
    !wget -q https://cv.snu.ac.kr/research/EDSR/benchmark.tar -O benchmark.tar
    !tar -xf benchmark.tar && rm benchmark.tar
    if os.path.exists('benchmark'):
        !mv benchmark/* . && rm -rf benchmark
    print("✅ 벤치마크 다운로드 완료")
else:
    print("✅ 벤치마크 데이터셋 이미 존재")

print(f"\n📁 벤치마크 구조:")
!ls -la

---
## 1.7 CATANet-L x2 원본 모델 PTL 변환

원본 CATANet-L x2 모델을 4가지 입력 해상도에 대해 PTL 파일로 변환합니다.
- 320x320, 640x640, 1280x1280, 1920x1920

**주의**: 메모리 사용량이 크므로 순차적으로 실행됩니다.

In [ ]:
%cd {project_dir}

print("="*60)
print("🔄 CATANet-L x2 원본 모델 PTL 변환")
print("="*60)

import gc
import torch

input_sizes = [320, 640, 1280, 1920]
weights_path = "weights/CATANet-L_x2.pth"

if not os.path.exists(weights_path):
    print(f"❌ 원본 가중치 파일이 없습니다: {weights_path}")
else:
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/CATANet-L_x2_{size}.ptl"
        
        if os.path.exists(output_path):
            print(f"✅ 이미 존재: {output_path}")
            continue
        
        !{python_exec} scripts/convert_to_ptl.py \
            --weights {weights_path} \
            --input_size {size} \
            --upscale 2 \
            --output {output_path}
        
        # 메모리 정리
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        print(f"✅ 완료: {output_path}")

print("\n" + "="*60)
print("📁 생성된 PTL 파일:")
!ls -lh weights/*.ptl 2>/dev/null || echo "PTL 파일 없음"

---
## 1.8 설정 완료 확인

In [ ]:
%cd {project_dir}

print("="*60)
print("✅ Setup 완료!")
print("="*60)
print(f"\n📁 프로젝트 경로: {project_dir}")
print(f"🐍 Python 실행: {python_exec}")
print(f"\n📂 프로젝트 구조:")
!ls -la

print(f"\n📂 weights 폴더:")
!ls -lh weights/ 2>/dev/null || echo "weights 폴더 없음"

# 다음 노트북에서 사용할 변수 저장
print(f"\n💡 다음 노트북(2_train.ipynb)에서 아래 변수를 사용하세요:")
print(f"   python_exec = '{python_exec}'")
print(f"   project_dir = '{project_dir}'")